# Data loading and initial cleansing

In [52]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
# import random

In [53]:
# Adjust settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
%matplotlib inline

In [54]:
# Load raw data into pandas dataframe
data_path = '../data/raw/'
flt_info = pd.read_csv(os.path.join(data_path, 'flight_information.csv'), index_col=0)
gnd_info = pd.read_csv(os.path.join(data_path, 'ground_information.csv'), index_col=0)

In [55]:
# Have a first look at flight data
flt_info.head()

,leg_no,fn_carrier,fn_number,dep_ap_sched,arr_ap_sched,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration,change_reason_code,dep_delay,Ac Type Code,trans_time,sched_trans_time,Crew Group,TLC_trans,crew_type_change,Sched Groundtime,Act Groundtime
0,272005011,East Carmen Airlines,EC3098,North Gregory,East Carmen,2019-06-01,00:40,2019-06-01,04:30,2019-06-01 00:43:00,2019-06-01 04:12:00,ECLBSX,other problem,3.0,320,0,0,Start,"['Jason Brown_nan_nan_nan_cp', 'Jacob Stein_na...",[],235.0,268.0
1,272009188,East Carmen Airlines,EC3146,Port Bobby,East Carmen,2019-06-01,02:05,2019-06-01,04:00,2019-06-01 01:58:00,2019-06-01 03:40:00,ECLWHX,no reason,-7.0,E95,0,0,Start,"['Scott Baker_nan_nan_nan_cp', 'Ruth Hoffman_n...",[],130.0,189.0
2,271984537,East Carmen Airlines,EC2840,Port Ryan,East Carmen,2019-06-01,02:20,2019-06-01,03:50,2019-06-01 02:24:00,2019-06-01 03:57:00,ECLGLX,other problem,4.0,DH4,0,0,Start,"['Cory Cooper_nan_nan_nan_ca', 'Morgan Foster_...",[],185.0,196.0
3,272025831,East Carmen Airlines,EC3306,Coleland,East Carmen,2019-06-01,02:25,2019-06-01,04:00,2019-06-01 02:28:00,2019-06-01 03:49:00,ECLBMX,other problem,3.0,320,0,0,Start,"['Anne Davis_nan_nan_nan_ca', 'Miss Amanda Har...",[],145.0,175.0
4,272006092,East Carmen Airlines,EC3114,East Melissaberg,East Carmen,2019-06-01,02:30,2019-06-01,04:15,2019-06-01 03:02:00,2019-06-01 04:43:00,ECLGNX,other problem,32.0,DH4,0,0,Start,"['Renee Fisher_nan_nan_nan_ca', 'Rebecca Castr...",[],35.0,21.0


In [56]:
# Combine departure and arrival to datetime and drop the initial columns
flt_info['flt_sched_dep'] = pd.to_datetime(flt_info.dep_sched_date.astype(str) + ' ' + flt_info.dep_sched_time.astype(str))
flt_info['flt_sched_arr'] = pd.to_datetime(flt_info.arr_sched_date.astype(str) + ' ' + flt_info.arr_sched_time.astype(str))
flt_info = flt_info.drop(columns=['dep_sched_date', 'dep_sched_time', 'arr_sched_date', 'arr_sched_time'])

In [57]:
# Have a first look at ground data
gnd_info.head()

,day_of_origin,ac_type,fn_number,ac_registration,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration
0,2019-06-01,DH4,EC3114,ECLGNX,35,East Melissaberg,East Carmen,Keithberg,2019-06-01 02:30:00,2019-06-01 04:15:00,2019-06-01 04:50:00,2019-06-01 05:40:00,35,272006092,272205857,26,NaN,26.0
1,2019-06-01,320,EC3316,ECLBIX,40,East Latashaview,East Carmen,Juliemouth,2019-06-01 03:15:00,2019-06-01 07:05:00,2019-06-01 07:50:00,2019-06-01 12:05:00,45,272027070,272029306,27,NaN,15.0
2,2019-06-01,321,EC3292,ECLBAX,45,New Jessica,East Carmen,South Nathaniel,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 08:20:00,2019-06-01 10:35:00,95,272024970,272022230,27,NaN,25.0
3,2019-06-01,E95,EC3420,ECLWFX,40,East Allisontown,East Carmen,Yoderburgh,2019-06-01 04:00:00,2019-06-01 04:35:00,2019-06-01 05:30:00,2019-06-01 07:05:00,55,272034482,271961698,18,NaN,11.0
4,2019-06-01,DH4,EC3400,ECLGBX,30,Port Courtneytown,East Carmen,West Ana,2019-06-01 04:00:00,2019-06-01 04:35:00,2019-06-01 05:10:00,2019-06-01 07:10:00,35,272033720,271984663,15,NaN,12.0


In [58]:
# Split ground data into inbound & outbound infromation and rename columns
gnd_inbound = gnd_info[['ac_type', 'ac_registration', 'dep_leg_inbound', 'arr_leg_inbound',
                 'sched_inbound_dep', 'sched_inbound_arr', 'leg_inbound']]
gnd_inbound.rename(columns={'ac_type': 'gnd_ac_type', 'ac_registration': 'gnd_ac_reg', 'dep_leg_inbound': 'gnd_dep_airpt',
                           'arr_leg_inbound': 'gnd_arr_airpt', 'sched_inbound_dep': 'gnd_sched_dep',
                           'sched_inbound_arr': 'gnd_sched_arr', 'leg_inbound': 'gnd_leg'}, inplace=True)
gnd_outbound = gnd_info[['ac_type', 'ac_registration', 'mingt', 'arr_leg_inbound', 'arr_leg_outbound',
                  'sched_outbound_dep', 'sched_outbound_arr', 'leg_outbound', 'sched_turnaround', 'catering_duration',
                  'cleaning_duration', 'pax_boarding_duration']]
gnd_outbound.rename(columns={'ac_type': 'gnd_ac_type', 'ac_registration': 'gnd_ac_reg', 'mingt': 'gnd_mingt',
                            'arr_leg_inbound': 'gnd_dep_airpt', 'arr_leg_outbound': 'gnd_arr_airpt',
                            'sched_outbound_dep': 'gnd_sched_dep', 'sched_outbound_arr': 'gnd_sched_arr',
                            'leg_outbound': 'gnd_leg', 'sched_turnaround': 'gnd_sched_tat', 'catering_duration': 'gnd_catering_dur',
                            'cleaning_duration': 'gnd_clean_dur', 'pax_boarding_duration': 'gnd_board_dur'},
                   inplace=True)
flt_info.rename(columns={'leg_no': 'flt_leg', 'fn_carrier': 'flt_carrier', 'fn_number': 'flt_fnum',
                   'dep_ap_sched': 'flt_dep_airpt', 'arr_ap_sched': 'flt_arr_airpt', 'm_offblockdt': 'flt_offblock',
                   'm_onblockdt': 'flt_onblock', 'ac_registration': 'flt_ac_reg', 'change_reason_code': 'flt_change_code',
                   'Ac Type Code': 'flt_ac_type', 'trans_time': 'flt_tt',
                   'sched_trans_time': 'flt_sched_tt', 'Crew Group': 'flt_crewg',
                   'TLC_trans': 'flt_TLC_trans', 'crew_type_change': 'flt_crewt_change', 'Sched Groundtime': 'flt_sched_gt',
                   'Act Groundtime': 'flt_act_gt'}, inplace=True)

In [59]:
# Merge flight data and ground data to one dataframe and adjust data type for columns
ground_combined = gnd_outbound.append(gnd_inbound, ignore_index=True)
df = pd.merge(flt_info, ground_combined, left_on='flt_leg', right_on='gnd_leg', how='left')
convert_dict = {'flt_leg': 'object',
                'flt_carrier': 'object',
                'flt_fnum': 'object',
                'flt_dep_airpt': 'object',
                'flt_arr_airpt': 'object',
                'flt_offblock': 'datetime64',
                'flt_onblock': 'datetime64',
                'flt_ac_reg': 'object',
                'flt_change_code': 'category',
                'dep_delay': 'float64',
                'flt_ac_type': 'category',
                'flt_tt': 'float64',
                'flt_sched_tt': 'float64',
                'flt_crewg': 'object',
                'flt_TLC_trans': 'object',
                'flt_crewt_change': 'object',
                'flt_sched_gt': 'float64',
                'flt_act_gt': 'float64',
                'flt_sched_dep': 'datetime64',
                'flt_sched_arr': 'datetime64',
                'gnd_ac_type': 'category',
                'gnd_ac_reg': 'object',
                'gnd_mingt': 'float64',
                'gnd_dep_airpt': 'object',
                'gnd_arr_airpt': 'object',
                'gnd_sched_dep': 'datetime64',
                'gnd_sched_arr': 'datetime64',
                'gnd_leg': 'object',
                'gnd_sched_tat': 'float64',
                'gnd_catering_dur': 'float64',
                'gnd_clean_dur': 'float64',
                'gnd_board_dur': 'float64'
}
df = df.astype(convert_dict)		

In [60]:
# Introduce arrival delay in minutes as dependent variable and drop rows without arr_delay
df['arr_delay'] = (df['flt_onblock'] - df['flt_sched_arr']) / pd.Timedelta(minutes=1)

In [61]:
# Drop all rows which are complete duplicates
# Normally there should be some action to understand why there are complete duplicates and curate this systematically
df[df.duplicated()].to_csv('../data/interim/duplicates.csv')
df.drop_duplicates(inplace=True, ignore_index=True)

In [62]:
# Storing pre-processed data to interim data location
df.to_csv('../data/interim/preprocessed.csv')
df.to_pickle('../data/interim/preprocessed.pkl')